<a href="https://colab.research.google.com/github/ekansh2004/Machine-Learning-Projects/blob/main/MedicalAbvEda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing the Required Python Packages
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from google.colab import drive
import chardet

In [ ]:
#Loading the data from the drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/train.csv'

Mounted at /content/drive


In [ ]:
# Loading the data in a dataframe.
textDF = pd.read_csv(file_path)

In [ ]:
#Checking the shape of Dataframe
textDF.shape

(3000000, 4)

In [ ]:
# The Data contains 3 million rows but System won't be able to work with such a huge dataset hence,
# will take the first 1 Million rows only for this project.
textDF.drop(textDF.index[1000000:], inplace = True)
textDF.shape

(1000000, 4)

In [ ]:
#Checking the first 5 rows of the dataframe
textDF.head(5)

,ABSTRACT_ID,TEXT,LOCATION,LABEL
0,14145090,velvet antlers vas are commonly used in tradit...,63,transverse aortic constriction
1,1900667,the clinical features of our cases demonstrate...,85,hodgkins lymphoma
2,8625554,ceftobiprole bpr is an investigational cephalo...,90,methicillinsusceptible s aureus
3,8157202,we have taken a basic biologic RPA to elucidat...,26,parathyroid hormonerelated protein
4,6784974,lipoperoxidationderived aldehydes for example ...,157,lipoperoxidation


In [ ]:
#Checking last 5 rows of the dataframe
textDF.tail(5)

,ABSTRACT_ID,TEXT,LOCATION,LABEL
999995,658615,in vivo flow p nmr spectroscopy of a microscop...,34,saturation transfer
999996,4297576,this study evaluated whether the number of gra...,52,severe aplastic anaemia
999997,500044,gmcsf is a major regulator of myelopoiesis rhG...,7,recombinant human gmcsf
999998,13583706,irritability is an aspect of the negative affe...,127,healthy comparison
999999,1540457,hypercalcemia is an uncommon complication of c...,27,congenital mesoblastic nephroma


In [ ]:
#Checking the summary statistics of the dataframe
textDF.describe(include = 'all')

,ABSTRACT_ID,TEXT,LOCATION,LABEL
count,1.000000e+06,1000000,1000000.000000,1000000
unique,NaN,939852,NaN,22490
top,NaN,purpose to investigate the effect of coordinat...,NaN,tibial nerve
freq,NaN,7,NaN,118
mean,7.027032e+06,NaN,76.750360,NaN
std,4.594329e+06,NaN,67.603551,NaN
min,6.000000e+00,NaN,0.000000,NaN
25%,3.057271e+06,NaN,22.000000,NaN
50%,6.381622e+06,NaN,60.000000,NaN
75%,1.082703e+07,NaN,114.000000,NaN


In [ ]:
#Checking the datatypes of dataframe columns
textDF.dtypes

,0
ABSTRACT_ID,int64
TEXT,object
LOCATION,int64
LABEL,object


In [ ]:
# Checking the unique Abstract_id to see if Abstract_id can be converted to Index
textDF['ABSTRACT_ID'].nunique()

939860

Hence, the Abstract_ID's are not all unique. So, lets check the duplicates in the dataset.

In [ ]:
duplicate = textDF[textDF.duplicated()]
duplicate.head(5)

,ABSTRACT_ID,TEXT,LOCATION,LABEL


Hence, none of the rows are duplicates.

In [ ]:
# Lets check for null values if any
textDF.isnull().values.any()

False

Thus, we don't have any Null values in the dataset.

In [ ]:
# Lets look at one row of the dataset in detail
pd.set_option('display.max_colwidth', 1)
textDF.head(1)

,ABSTRACT_ID,TEXT,LOCATION,LABEL
0,14145090,velvet antlers vas are commonly used in traditional chinese medicine and invigorant and contain many PET components for health promotion the velvet antler peptide svap is one of active components in vas based on structural study the svap interacts with tgfÎ² receptors and disrupts the tgfÎ² pathway we hypothesized that svap prevents cardiac fibrosis from pressure overload by blocking tgfÎ² signaling SDRs underwent TAC tac or a sham operation T3 one month rats received either svap mgkgday or vehicle for an additional one month tac surgery induced significant cardiac dysfunction FB activation and fibrosis these effects were improved by treatment with svap in the heart tissue tac remarkably increased the expression of tgfÎ² and connective tissue growth factor ctgf ROS species C2 and the phosphorylation C2 of smad and ERK kinases erk svap inhibited the increases in reactive oxygen species C2 ctgf expression and the phosphorylation of smad and erk but not tgfÎ² expression in cultured cardiac fibroblasts angiotensin ii ang ii had similar effects compared to tac surgery such as increases in Î±smapositive CFs and collagen synthesis svap eliminated these effects by disrupting tgfÎ² IB to its receptors and blocking ang iitgfÎ² downstream signaling these results demonstrated that svap has antifibrotic effects by blocking the tgfÎ² pathway in CFs,63,transverse aortic constriction


As per dataset specifications, location column signifies the word count after which the Abbreviation occurs and its Label is provided in Lable column

In [ ]:
# Lets check the Abbreviations of first 10 rows of the dataset alongwith their labels
split_text = [ t.split(' ') for t in textDF[:10]['TEXT']]
label = [t for t in textDF[:10]['LABEL']]
location = [t for t in textDF[:10]['LOCATION']]

In [ ]:
for i in range(0,10):
    print(label[i], ' -- ', split_text[i][location[i]])

transverse aortic constriction  --  TAC
hodgkins lymphoma  --  HD
methicillinsusceptible s aureus  --  MSSA
parathyroid hormonerelated protein  --  PTHrP
lipoperoxidation  --  LPO
hepatitis g virus  --  HGV
radical neck dissection  --  RND
amplified spontaneous emission  --  ASE
portal blood  --  HPB
western medicine  --  WM


From the above analysis, the relationship between Location, Label and Text columns are clearly visible.

Let us again check the number of unique ABSTRACT_ID in Dataset


In [ ]:
# Checking the unique Abstract_id
textDF['ABSTRACT_ID'].nunique()

939860

In [ ]:
# Checking the shape of the Dataset
textDF.shape

(1000000, 4)

It can be seen here that there are some Abstract_ID's which are not unique. Lets find those abstracts and check what is the main differences

In [ ]:
duplicate = textDF[textDF['ABSTRACT_ID'].duplicated(keep = False)]

In [ ]:
duplicate.sort_values(by = ['ABSTRACT_ID']).head(5)

,ABSTRACT_ID,TEXT,LOCATION,LABEL
11397,6,a doubleblind T0 with intraindividual comparisons was carried out to investigate the effects of mg of ralphahydroxyisopropylalphahtropanium bromidetropate sch mg sch mg OX mg oxazepam and placebo with p.o. in randomized CS on gastric juice volume amount of acid concentration and ph values in healthy volunteers the secretion parameters were measured during a h basal period and a h stimulation period the gastric juice was obtained in min portions via stomach tube stimulation was effected by mugkgh PG via drip infusion the friedman test was used for the comparative statistical evaluation and individual comparisons were carried out by means of the wilcoxon test pairdifferences rank the results show that sch and sch OX were equal in effect on basal and stimulated secretion volume as compared with PL it was not possible to establish an effect on secretion volume for oxazepam CT sch and sch oxazepam were found to be equipotent in reducing the amount of basal acid while oxazepam reduced this quantity only during the first min of basal secretion none of the three AS S9 was capable of inhibiting the stimulated acid although both sch preparations produced a clear trend towards lowered mean values during the basal secretion period all three test S9 had an inhibiting action on acid concentration but none of them had a significant effect during the stimulation period the ph value was savely increased only by sch and sch OX and this even only during the basal period the results are discussed,234,oxazepam
331391,6,a doubleblind T0 with intraindividual comparisons was carried out to investigate the effects of mg of ralphahydroxyisopropylalphahtropanium bromidetropate sch mg sch mg OX mg oxazepam and placebo with p.o. in randomized CS on gastric juice volume amount of acid concentration and ph values in healthy volunteers the secretion parameters were measured during a h basal period and a h stimulation period the gastric juice was obtained in min portions via stomach tube stimulation was effected by mugkgh PG via drip infusion the friedman test was used for the comparative statistical evaluation and individual comparisons were carried out by means of the wilcoxon test pairdifferences rank the results show that sch and sch OX were equal in effect on basal and stimulated secretion volume as compared with PL it was not possible to establish an effect on secretion volume for oxazepam CT sch and sch oxazepam were found to be equipotent in reducing the amount of basal acid while oxazepam reduced this quantity only during the first min of basal secretion none of the three AS S9 was capable of inhibiting the stimulated acid although both sch preparations produced a clear trend towards lowered mean values during the basal secretion period all three test S9 had an inhibiting action on acid concentration but none of them had a significant effect during the stimulation period the ph value was savely increased only by sch and sch OX and this even only during the basal period the results are discussed,112,oxazepam
627471,75,P2 fractions p derived from guineapig CBF cortex were incubated in the presence of mm kcl in a krebsglucose medium torpedo marmorata electric organs were stimulated electrically in vivo at pulsessec for min by electrodes placed on the electric lobe synaptic LDV were isolated from each source and the phospholipid compositions analysed and compared with LDV from unstimulated controls lysophosphatidylcholine was the only lysophosphoglyceride demonstrable in the SVs from either source and its low levels did not increase as a result of chemical or electircal stimulation in each case there was a close similarity of the phospholipid distributions in the vesicles taken from control and stimulated samples control experiments indicated extensive decreases in the acetylcholine content of the LDV from the stimulated electric organ and smaller decreases in the acetylcholine content of the SVs from stimulated c

So, based on the above results for duplicates, it can be seen that a single Text might contain more than 1 Abbreviation at diffrent places. Thus, multiple row for multiple Abbreviations are present.

Let's save the above trainDF in a csv file for further use

In [ ]:
save_path = '/content/drive/My Drive/cleaned_train.csv'
textDF.to_csv(save_path, index=False)
print('cleaned_train saved')

cleaned_train saved
